# Desafio: Construindo um modelo de Regressão para Marketing

# Contexto - Introdução

Uma empresa está investindo mensalmente em plataformas de publicidade online, como Youtube, Facebook e newspaper, para a prospecção de leads (pessoas interessadas em seus produtos). A fim de acompanhar o desempenho desses investimentos, a empresa registra todos os gastos com publicidade e todos os retornos de vendas gerados a partir desses investimentos.

Para **entender** melhor **a relação entre as variáveis** presentes nesses registros e **identificar os fatores que mais impactam** na geração de leads, a empresa solicitou a análise de um especialista em dados. **Além disso, a empresa busca criar um modelo de predição** de valores para estimar o retorno de vendas que pode ser gerado a partir de um determinado investimento em publicidade.

# Sobre os dados

A tabela contém informações dos investimentos feitos pelo youtube, facebook, newspaper e também a quantidade de cada. 

| Coluna    | Descrição              |
|-----------|------------------------|
| youtube   | Investimento youtube   |
| facebook  | Investimento facebook  |
| newspaper | Investimento newspaper |
| sales     | Valor das vendas       |

# Importando as Bibliotecas

In [1]:
#Manipulação de Dados
import pandas as pd
import numpy as np

#Visualização
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objs as go
import plotly.figure_factory as ff

#Display
import warnings
import handcalcs.render
from IPython.display            import Markdown

# Funções Auxiliares e Configurações

In [47]:
warnings.filterwarnings("ignore")
pd.options.display.float_format = '{:.2f}'.format

def df_informations(df):
    # Cria um Dataframe com as informações relativas ao Dataset
    df_info = pd.DataFrame({'Not Null': df.notnull().sum(),
                            'Null': df.isnull().sum(),
                            'Perce Null': df.isnull().sum() / len(df),
                            'Dtype': df.dtypes
                })
    
    # Cria outro Dataframe com todos os types do df
    df_dtype = df_info['Dtype'].value_counts().reset_index()
    df_dtype.columns = ['Dtype', 'Count']
    df_dtype['Perce'] = round(df_dtype['Count'] / df_dtype['Count'].sum(), 2)
    
    # Verificando se tem linhas duplicadas
    duplicado = df.duplicated().sum()
    mensagem = 'Não possui linhas duplicadas.' if duplicado == 0 else f"Tem {duplicado} linhas duplicadas, o que representa {round(duplicado/len(df)*100, 2)}% do total de linhas."

    # Texto automático das dimensões do df
    text = f'Dataset tem {df.shape[0]} linhas e {df.shape[1]} colunas. {mensagem} Sobre o dataset, temos:'

    # Colore os Dataframes
    df_info = df_info.style.background_gradient(cmap='jet', subset=['Perce Null']).format({'Perce Null': '{:.2%}'})
    df_dtype = df_dtype.style.background_gradient(cmap='YlGn', subset=['Perce']).format({'Perce': '{:.2%}'})

    # Texto final com Markdown
    display(Markdown("<H3 style='text-align:left;float:lfet;'>Informações sobre o Dataset"))
    display(Markdown(f'<H5> {text}'))
    display(df_info)
    display(Markdown("<H3 style='text-align:left;float:lfet;'>Sobre Dtypes, temos:"))
    display(df_dtype)
    display(Markdown("<H3 style='text-align:left;float:lfet;'>Estatística Descritiva"))
    display(df.describe(percentiles=[0.01, 0.25, 0.5, 0.75, 0.99]).T)

def plot_distribution_and_boxplot(df):
    # Obtém a lista de colunas numéricas
    num_cols = df.select_dtypes(include=[np.number]).columns
    
    # Cria a lista de títulos para os subplots
    subplot_titles = [title for col in num_cols for title in [f"Histograma - {col}", f"Boxplot - {col}"]]
    
    # Cria a figura com subplots
    fig = sp.make_subplots(rows=len(num_cols), cols=2, subplot_titles=subplot_titles)
    
    # Define cores
    colors = ['blue', 'red', 'green', 'purple', 'orange', 'brown', 'pink', 'yellow', 'cyan', 'magenta', 
              'lime', 'lavender', 'maroon', 'navy', 'olive', 'teal', 'aqua', 'fuchsia', 'gray', 'silver']
    
    # Adiciona os gráficos à figura para cada coluna numérica
    for i, col in enumerate(num_cols, start=1):
        # Define a cor usando a lógica de ciclagem
        color = colors[i % len(colors)]
        
        # Adiciona o histograma
        fig.add_trace(go.Histogram(x=df[col], nbinsx=40, marker_color=color, 
                                    showlegend=False),
                      row=i, col=1)
        
        # Adiciona o boxplot
        fig.add_trace(go.Box(y=df[col], marker_color=color, showlegend=False),
                      row=i, col=2)
    
    # Configura o layout
    fig.update_layout(height=300*len(num_cols), title_text="Distribuição e Boxplot para cada variável numérica")
    fig.show()


# Importando o Dataset

In [34]:
df = pd.read_csv('MKT.csv')
df

,youtube,facebook,newspaper,sales
0,84.72,19.20,48.96,12.60
1,351.48,33.96,51.84,25.68
2,135.48,20.88,46.32,14.28
3,116.64,1.80,36.00,11.52
4,318.72,24.00,0.36,20.88
...,...,...,...,...
166,45.84,4.44,16.56,9.12
167,113.04,5.88,9.72,11.64
168,212.40,11.16,7.68,15.36
169,340.32,50.40,79.44,30.60


# Etapas de Desenvolvimento

Para te ajudar nesse processo, detalhar o processo nas etapas a seguir:

## Etapa 01) Análise Descritiva
Esta etapa consiste em explorar os dados do dataset para **compreender melhor as variáveis e identificar problemas**. Para isso, é recomendado utilizar a biblioteca **Pandas** para importar e manipular os dados e realizar cálculos estatísticos, além das bibliotecas de visualização.

É importante investigar o tipo de dado em cada variável, os valores e a distribuição dos
dados. Ao final, espera-se ter uma interpretação sólida dos dados para avançar para a
próxima etapa

### Análise Descritiva do Dataset

In [42]:
df_informations(df)

<H3 style='text-align:left;float:lfet;'>Informações sobre o Dataset

<H5> Dataset tem 171 linhas e 4 colunas. Não possui linhas duplicadas. Sobre o dataset, temos:

,Not Null,Null,Perce Null,Dtype
youtube,171,0,0.00%,float64
facebook,171,0,0.00%,float64
newspaper,171,0,0.00%,float64
sales,171,0,0.00%,float64


<H3 style='text-align:left;float:lfet;'>Sobre Dtypes, temos:

,Dtype,Count,Perce
0,float64,4,100.00%


<H3 style='text-align:left;float:lfet;'>Estatística Descritiva

,count,mean,std,min,1%,25%,50%,75%,99%,max
youtube,171.00,178.02,102.45,0.84,6.01,91.08,179.76,262.98,351.73,355.68
facebook,171.00,27.67,17.91,0.00,0.44,11.70,26.76,43.68,59.28,59.52
newspaper,171.00,35.24,24.90,0.36,1.75,13.74,31.08,50.88,103.42,121.08
sales,171.00,16.92,6.31,1.92,5.60,12.54,15.48,20.82,30.85,32.40


As informações **.describe** podem ser úteis para entender a dispersão e a tendência central dos dados.

- <font color='chartreuse'>Média</font>: Representa a tendência central dos dados;
- <font color='chartreuse'>Desvio Padrão</font>: Indica a dispersão, sugerindo uma variabilidade dos valores;
- <font color='chartreuse'>Quartis</font>: Fornecem informações sobre a distribuição dos dados ao longo de diferentes partes; e
- <font color='chartreuse'>Valor Mínimo e Máximo</font>: Indicam a faixa em que os dados estão concentrados.

Logo, podemos observar que:

1. **Variável "youtube":**
    * A média de investimento do YouTube é de aproximadamente 178.02, o que nos mostra que é o que mais investe.
    * O desvio padrão é relativamente alto (102.45), o que sugere que os dados do YouTube apresentam uma dispersão considerável em relação à média.
    * Os quartis indicam que 25% dos dados estão abaixo de 91.08, 50% estão abaixo de 179.76 e 75% estão abaixo de 262.98.
    * O valor mínimo é de 0.84 e o valor máximo é de 355.68, mostrando que a faixa de visualizações varia de forma considerável.
<br></br>
1. **Variável "facebook":**
    * A média (27.67) de investimento do Facebook é aproximadamente 6.5x menor em relação ao Youtube.
    * O desvio padrão é de 17.91, indicando uma dispersão alta (64.72%) em relação à média.
    * Os quartis mostram que 25% dos dados estão abaixo de 11.70, 50% estão abaixo de 26.76 e 75% estão abaixo de 43.68.
    * O valor mínimo é 0.00 e o valor máximo é 59.52, mostrando a faixa de valores para a métrica do Facebook.
<br></br>
1. **Variável "newspaper":**
    * A média (35.24) do newspaper é a segunda maior ficando atrás apenas do Youtube.
    * O desvio padrão é de 24.90, indicando uma dispersão relativamente alta (70.65%) em relação à média.
    * Os quartis mostram que 25% dos dados estão abaixo de 13.74, 50% estão abaixo de 31.08 e 75% estão abaixo de 50.88.
    * O valor mínimo é 0.36 e o valor máximo é 121.08, indicando uma ampla faixa de valores.
<br></br>
1. **Variável "sales":**
    * A média para os dados de vendas é de aproximadamente 16.92.
    * O desvio padrão é de 6.31, sugerindo uma dispersão moderada (37.29%) em relação à média.
    * Os quartis mostram que 25% dos dados estão abaixo de 12.54, 50% estão abaixo de 15.48 e 75% estão abaixo de 20.82.
    * O valor mínimo é 1.92 e o valor máximo é 32.40, mostrando a faixa de valores para as vendas.

### Análise Gráfica

In [48]:
plot_distribution_and_boxplot(df)

## Etapa 02) Análise Exploratória
Neta etapa iremos explorar mais a fundo os dados, **identificando relações entre as variáveis e descobrindo padrões relevantes**. Para isso, utilize técnicas de visualização de dados e análises estatísticas, buscando possíveis correlações e identificando possíveis outliers ou desvios da normalidade.

### Definição de Correlação

A correlação é uma medida estatística que *avalia a relação entre duas variáveis*. Ela indica a direção e a intensidade dessa relação, ou seja, se as variáveis se movem em conjunto (correlação positiva) ou de forma oposta (correlação negativa), e o quão forte essa relação é.

A força da correlação pode ser definida com base no valor do coeficiente de correlação, que varia de -1 a +1. Aqui estão as interpretações comuns para determinar a força da correlação:

1. <font color='yellow'>Correlação fraca:</font>
    * Quando o coeficiente de correlação está próximo de 0, a correlação é considerada fraca.
    * Seja ela positiva ou negativa, a relação entre as variáveis é considerada fraca se o coeficiente de correlação estiver próximo de zero (por exemplo, entre -0,3 e 0,3).
    * Nesse caso, as variáveis têm uma associação limitada e seus movimentos não são consistentes.
<br></br>
2. <font color='orange'>Correlação média:</font>
    * Uma correlação é considerada média quando o coeficiente de correlação está em torno de -0,5 a -0,3 ou de 0,3 a 0,5.
    * A relação entre as variáveis é moderada e mostra algum grau de consistência em seus movimentos.
    * Uma correlação média indica que as variáveis têm alguma influência mútua, mas não é uma relação forte.
<br></br>
3. <font color='chartreuse'>Correlação forte:</font>
    * A correlação é considerada forte quando o coeficiente de correlação está próximo de -1 ou 1.
    * Uma correlação positiva forte (próxima de +1) indica que as variáveis estão fortemente relacionadas e tendem a se mover na mesma direção.
    * Uma correlação negativa forte (próxima de -1) indica que as variáveis estão fortemente relacionadas, mas se movem em direções opostas.
    * Nesses casos, as variáveis têm uma associação consistente e os movimentos de uma variável estão altamente relacionados aos movimentos da outra.


É importante destacar que a força da correlação pode variar de acordo com o contexto e o domínio dos dados. Além disso, a correlação não implica causalidade direta, ou seja, apenas porque duas variáveis estão correlacionadas, não significa que uma causa a outra.

### Matriz de Correlação

In [7]:
# Matriz de Correlação
correlation_matrix = df.corr()

# Cria o gráfico de heatmap
fig = go.Figure(data=go.Heatmap(
                   z=correlation_matrix,
                   x=correlation_matrix.columns,
                   y=correlation_matrix.columns,
                   colorscale='Viridis'))

# Adiciona anotações
annotations = []
for i, row in enumerate(correlation_matrix.values):
    for j, val in enumerate(row):
        annotations.append(go.layout.Annotation(text=str(round(val, 2)), x=correlation_matrix.columns[j], y=correlation_matrix.columns[i], 
                                                showarrow=False, font=dict(color='white')))

# Atualiza layout
fig.update_layout(title='Matriz de Correlação',
                  annotations=annotations,
                  height=500, width=600)

fig.show()


Explicado o que é correlação, podemos observar que:

1. A correlação entre "youtube" e "sales" é de 0.78, o que indica uma correlação positiva forte. Isso sugere que há uma forte relação entre o investimento no YouTube e as vendas. Aumento nas vendas tendem a estar associados ao aumento de investimento no YouTube.

2. A correlação entre "facebook" e "sales" é de 0.60, o que indica uma correlação positiva forte. Isso sugere que há uma relação forte entre o investimento no Facebook e as vendas. Aumentos nas vendas tendem a estar associados ao aumento de investimento no Facebook.

3. A correlação entre "newspaper" e "sales" é de 0.25, o que indica uma correlação positiva fraca. Isso sugere que há uma relação fraca entre o investimento em newspaper e as vendas. A influência dos newspaper nas vendas é limitada, pois a correlação é relativamente baixa.

### Scatter plot: Sales vs Investimento

In [8]:
# Reestrutura os dados para um formato longo
df_melted = df.melt(id_vars='sales', var_name='platform', value_name='investment')

# Cria um dicionário com os símbolos para cada plataforma
symbols = {'youtube': 'circle', 'facebook': 'diamond', 'newspaper': 'square'}

# Cria o gráfico de dispersão
fig = px.scatter(df_melted, x='investment', y='sales', color='platform', 
                 symbol=df_melted['platform'].map(symbols), 
                 title='Scatter plot: Sales vs Investimento',
                 labels={'investment':'Investimento', 'sales':'Sales'}, 
                 hover_data=['platform', 'investment', 'sales'])

fig.show()


Analisando o Scatter plot é possível observar que é necessário desprender ou investir muito mais dinheiro no Youtube para ter o mesmo retorno que as demais plataformas. Em contra partida o Facebook e Newspaper possuem uma distribuição semelhante, onde a relação Sales vs Investimento é mais vantajosa, sendo necessário despender menos dinheiro para um retorno maior.

### Analisando a Eficiência da relação Sales vs Investimento

Uma maneira de avaliar a eficiência do investimento em marketing é calcular a taxa de retorno do investimento (ROI), que é uma métrica popular para avaliar a eficiência do investimento. O ROI é calculado como:

**ROI = (Retorno do Investimento - Custo do Investimento) / Custo do Investimento**

Sendo que:
* Retorno do Investimento = Vendas (sales)
* Custo do Investimento = Investimento nas Plataformas

In [51]:
df['ROI_youtube'] = (df['sales'] - df['youtube']) / df['youtube']
df['ROI_facebook'] = (df['sales'] - df['facebook']) / df['facebook']
df['ROI_newspaper'] = (df['sales'] - df['newspaper']) / df['newspaper']
df

,youtube,facebook,newspaper,sales,ROI_youtube,ROI_facebook,ROI_newspaper
0,84.72,19.20,48.96,12.60,-0.85,-0.34,-0.74
1,351.48,33.96,51.84,25.68,-0.93,-0.24,-0.50
2,135.48,20.88,46.32,14.28,-0.89,-0.32,-0.69
3,116.64,1.80,36.00,11.52,-0.90,5.40,-0.68
4,318.72,24.00,0.36,20.88,-0.93,-0.13,57.00
...,...,...,...,...,...,...,...
166,45.84,4.44,16.56,9.12,-0.80,1.05,-0.45
167,113.04,5.88,9.72,11.64,-0.90,0.98,0.20
168,212.40,11.16,7.68,15.36,-0.93,0.38,1.00
169,340.32,50.40,79.44,30.60,-0.91,-0.39,-0.61


In [52]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
youtube,171.00,178.02,102.45,0.84,91.08,179.76,262.98,355.68
facebook,171.00,27.67,17.91,0.00,11.70,26.76,43.68,59.52
newspaper,171.00,35.24,24.90,0.36,13.74,31.08,50.88,121.08
sales,171.00,16.92,6.31,1.92,12.54,15.48,20.82,32.40
ROI_youtube,171.00,-0.84,0.22,-0.96,-0.92,-0.90,-0.85,1.29
ROI_facebook,171.00,inf,NaN,-0.96,-0.52,-0.32,0.32,inf
ROI_newspaper,171.00,0.41,4.70,-0.90,-0.68,-0.51,0.10,57.00


#### Corrigindo o inf do ROI_facebook

Após análise dos dados foi observado que na linha 98 do dataset a variável facebook é igual a 0, sendo assim quando calculamos o ROI, temos:

**ROI = (10.56-0)/0**

Matematicamente não é possível dividir um número por zero, então para corrigir esse problema vamos adotar uma premissa.

In [53]:
df.loc[98]

youtube         96.24
facebook         0.00
newspaper       11.04
sales           10.56
ROI_youtube     -0.89
ROI_facebook      inf
ROI_newspaper   -0.04
Name: 98, dtype: float64

Como premissa vamos adotar que o 0 será a média.

In [54]:
df['ROI_facebook'][98] = df['facebook'].mean()

In [55]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
youtube,171.00,178.02,102.45,0.84,91.08,179.76,262.98,355.68
facebook,171.00,27.67,17.91,0.00,11.70,26.76,43.68,59.52
newspaper,171.00,35.24,24.90,0.36,13.74,31.08,50.88,121.08
sales,171.00,16.92,6.31,1.92,12.54,15.48,20.82,32.40
ROI_youtube,171.00,-0.84,0.22,-0.96,-0.92,-0.90,-0.85,1.29
ROI_facebook,171.00,0.62,3.48,-0.96,-0.52,-0.32,0.32,28.00
ROI_newspaper,171.00,0.41,4.70,-0.90,-0.68,-0.51,0.10,57.00
